In [ ]:
#@title Connecting to google drive (Before Running Main Code Connect to Drive)🥇
from google.colab import drive
drive.mount('/content/drive/')


from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


# importing libraries
import pandas as pd
import shutil
import os
import datetime 
import numpy as np
#
import re

In [ ]:
#@title Updating Descriptions with Nutritional values in New Descriptions Column

Menu_sheet_id_where_New_descriptions_need_to_be_updated='1NFCWTp1q9Q0LYhdBuIhyeYH1p0xcFrHBXjQZtEhME7M'  # paste the sheet id here

Menu_Sheet_tab_name = '08_Menu_Item_Fixed_Name_V2' # paste ur sheet tab name here





# ******************************************************************************* MAIN CODE ***********************************************************************************
worksheet2 = gc.open_by_key(f'{Menu_sheet_id_where_New_descriptions_need_to_be_updated}').worksheet(f'{Menu_Sheet_tab_name}')
rows = worksheet2.get_all_values()
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

new_descriptions = []
df['Description']=df['Description'].str.replace(".","",regex=True)

for _, row in df.iterrows():
    description = row['Description']

    kcal = str(row['Kcal'])
    carbs = str(row['Carbs (g)'])
    fat = str(row['Fat (g)'])
    Protien = str(row['Protien (g)'])
    fiber = str(row['Fiber'])


    values  = [f"Kcal:{kcal:}"] if kcal != '0' and kcal != '' else []
    values += [f"Carbs (g):{carbs:}"] if carbs != '0' and carbs != '' else []
    values += [f"Fat (g):{fat:}"] if fat != '0' and fat != '' else []
    values += [f"Protien (g):{Protien:}"] if Protien != '0' and Protien != '' else []
    values += [f"Fiber:{fiber:}"] if fiber != '0' and fiber != '' else []
  
    # create the new description by joining the values with commas
    
    if values:
        new_description = "[Energy: "+", ".join(values)
        new_description += " ]"
        if description:
            new_description = f"{description}. {new_description}"
    else:
        new_description = description

    new_descriptions.append(new_description)
    

# add the new description column to the DataFrame
df['New description'] = new_descriptions

# update the worksheet with the updated DataFrame
import gspread_dataframe as gd
gd.set_with_dataframe(worksheet2, df)


## Ignore Below Code (Extra)

In [ ]:
#@title click

naming_standard_file_sheet_id='1NFCWTp1q9Q0LYhdBuIhyeYH1p0xcFrHBXjQZtEhME7M'
Menu_file_to_be_updated_sheet_id='18A9Rqt-YBPdVw8ee6-DE_hpajrtUYWW-VLLh-8KuEQQ'




worksheet1 = gc.open_by_key(f'{Menu_file_to_be_updated_sheet_id}').worksheet('Menu_Item_Report_28547_2023_03_28_02_22_07')
rows=worksheet1.get_all_values()
Menu=pd.DataFrame.from_records(rows[1:],columns=rows[0])

worksheet2 = gc.open_by_key(f'{naming_standard_file_sheet_id}').worksheet('08_Menu_Item_Fixed_Name_V2')
rows=worksheet2.get_all_values()
naming=pd.DataFrame.from_records(rows[1:],columns=rows[0])

# Get the list of unique Online_Names from Menu
online_names = Menu['Online_Name'].unique()

descriptions = {}


for online_name in online_names:

    matching_row = naming[naming['Fixed Name '] == online_name]
    if not matching_row.empty:
        description = matching_row['Description'].values[0]
        descriptions[online_name] = description

# Update the 'Description' column in the 'Menu' worksheet using the 'update' method
updated_values = []
for index, row in Menu.iterrows():
    if row['Online_Name'] in descriptions:
        description = descriptions[row['Online_Name']]
        Menu.at[index, 'Description'] = description
    updated_values.append([row['Description']])


import gspread_dataframe as gd
wg=gc.open_by_key('18A9Rqt-YBPdVw8ee6-DE_hpajrtUYWW-VLLh-8KuEQQ').worksheet('Output_file')

# new_menu=pd.DataFrame(Menu)
# gd.set_with_dataframe(wg,new_menu)

In [ ]:
#@title oclick category
import gspread_dataframe as gd

# Create Table1
wc1 = gc.open_by_key("1UXjL3dQcsePcjigqK1SWs9P0OAn65FG2mPdbJn6VIA8").worksheet('Category_Report_2023_04')
rows=wc1.get_all_values()

to_be_updated=pd.DataFrame.from_records(rows[1:],columns=rows[0])


wc2 = gc.open_by_key("1NFCWTp1q9Q0LYhdBuIhyeYH1p0xcFrHBXjQZtEhME7M").worksheet('07_Category_Fixed_Name')
rows1=wc2.get_all_values()

get_data=pd.DataFrame.from_records(rows1[1:],columns=rows1[0])


# Iterate over rows in Table1
for index, row in to_be_updated.iterrows():
    # Get the 'Online Display Name' value for the current row
    online_display_name = row['New Online Display Name']
    
    # Check if the 'Online Display Name' value is in the 'Category Name with Portion Sizes' column of Table2
    if any(get_data['Category Name with Portion Sizes'].str.contains(online_display_name)):
        # Get the corresponding 'Final Prefix' value from get_data and update the 'final_prefix' column in to_be_updated
        standard_name = get_data.loc[get_data['Category Name with Portion Sizes'].str.contains(online_display_name), 'Cuisine Prefix'].iloc[0]
        to_be_updated.loc[index, 'Cuisine Prefix 2'] = standard_name
    # else:
    #     to_be_updated.loc[index, 'New_Display Name'] = "Not_Present"


cell_list = wc1.range('J2:J500')

cell_values = to_be_updated['Cuisine Prefix 2'].tolist()

for i, val in enumerate(cell_values):  #gives us a tuple of an index and value
    cell_list[i].value = val    #use the index on cell_list and the val from cell_values

wc1.update_cells(cell_list)



# gd.set_with_dataframe(wc1,to_be_updated)

<ipython-input-45-c1d97e46fc92>:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if any(get_data['Category Name with Portion Sizes'].str.contains(online_display_name)):


{'spreadsheetId': '1UXjL3dQcsePcjigqK1SWs9P0OAn65FG2mPdbJn6VIA8',
 'updatedRange': 'Category_Report_2023_04!J2:J500',
 'updatedRows': 499,
 'updatedColumns': 1,
 'updatedCells': 499}

In [ ]:
#@title giving prefix of if len < 1

c=0
for index,row in df.iterrows():
  r=str(row['fo'])
  if r=="":
    df.loc[index,'fo']=r
  elif len(r)!=2:
    c+=1
    df.loc[index,'fo']='0'+r
    print(r)


1
2
3


In [ ]:
new_description

'[Energy: Kcal:, Carbs (g):, Fat (g):, Protien (g):, Fiber: ]'

In [ ]:
#@title iiclick

worksheet1 = gc.open_by_key('1PIYblBA25aPWGb2l4Z_SXV-8j2XRr8aMPNc0Ib5S6Vw').worksheet('Item_AreawisePrice_28547')
rows=worksheet1.get_all_values()
menu_file=pd.DataFrame.from_records(rows[1:],columns=rows[0])

worksheet2 = gc.open_by_key('1NFCWTp1q9Q0LYhdBuIhyeYH1p0xcFrHBXjQZtEhME7M').worksheet('08_Menu_Item_Fixed_Name_V2')
rows=worksheet2.get_all_values()
naming=pd.DataFrame.from_records(rows[1:],columns=rows[0])

# Get the list of unique Online_Names from menu_file
unique_names = menu_file['ItemName'].unique()

# Loop through the unique names and update the corresponding rows in menu_file
for name in unique_names:
    mask = menu_file['ItemName'] == name
    if name in naming['Fixed Name '].values:
        description = naming.loc[naming['Fixed Name '] == name, 'New description'].iloc[0]
        menu_file.loc[mask, 'ItemDescription'] = description

# Update the worksheet with the modified menu_file
import gspread_dataframe as gd

gd.set_with_dataframe(worksheet1, menu_file)


In [ ]:
#@title Click1

worksheet2 = gc.open_by_key('1UgJNSDtStc1kTr1XNYcgB3mLfL-EM97mYOGIpdMO9i4').worksheet('Sheet4')
rows=worksheet2.get_all_values()
df=pd.DataFrame.from_records(rows[1:],columns=rows[0])

new_descriptions = []
for i in range(len(df)):
    # Skip the row if any of the columns Kcal, Carbs (g), Fat (g), Protien (g), Fiber contains 0
    if '0' in [df['Kcal'][i], df['Carbs (g)'][i], df['Fat (g)'][i], df['Protien_(g)'][i], df['Fiber'][i]]:
        new_descriptions.append('')
    else:
        new_description = "{} Kcal - {}, Carbs (g) - {}, Fat (g) - {}, Protien_(g) - {}, Fiber - {}".format(
            df['Description'][i], df['Kcal'][i], df['Carbs (g)'][i], df['Fat (g)'][i], df['Protien_(g)'][i], df['Fiber'][i])
        new_descriptions.append(new_description)

# Update the DataFrame with the new values
df['New description'] = new_descriptions

# Update the worksheet with the updated DataFrame
worksheet2.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1UgJNSDtStc1kTr1XNYcgB3mLfL-EM97mYOGIpdMO9i4',
 'updatedRange': 'Sheet4!A1:IA463',
 'updatedRows': 463,
 'updatedColumns': 235,
 'updatedCells': 108805}

In [ ]:
df.head(2)

,Fixed Name,Description,Short_Code,Short_Code_2,Sap_Code,HSN_Code,Category_online_display,Attributes,Goods_Services,"GST% (Allowed only 5,12,18,28)",...,Addon_Group_Selection,Addon_Group_Min,Addon_Group_Max,New description,Kcal,Carbs (g),Fat (g),Protien (g),Fiber,FoodAmount / ServingSize:
0,Curd Rice,Classic creamy curd rice combination tempered ...,253,,,,Rice,veg,services,5,...,,,,Classic creamy curd rice combination tempered ...,207,37.6,3.2,6.1,1.8,
1,Boiled Egg (1 Pcs),,204,,,,Egg Dry & Gravies,egg,services,5,...,,,,,72,0.4,4.8,6.3,0,


In [ ]:
#worksheet = gc.open_by_key('1ZGSAX5BEHxoxzKaL0jJPchBFk7L2RDGL8Ie6W0CmkI4').worksheet('08_Menu_Item_Fixed_Name_V2')
worksheet = gc.open_by_key('1ZGSAX5BEHxoxzKaL0jJPchBFk7L2RDGL8Ie6W0CmkI4').sheet1

rows=worksheet.get_all_values()
cate=pd.DataFrame.from_records(rows[1:],columns=rows[0])

#worksheet = gc.open_by_key('1ikO3TyarCdjLgWrfukhuOOxjLB5C_MtCsfPPGbgbc8c').sheet1
rows1=worksheet3.get_all_values()
#df12=pd.DataFrame.from_records(rows1[1:],columns1=rows[0])

In [ ]:
table.groupby.cate(['Short_Code_2',	'Sap_Code'	,'HSN_Code',	'Category_online_display'])[['Description']].sum().to_excel('Jan_data_Swigy-2.xlsx')

In [ ]:
cate.head(2)

,Fixed Name,Description,Short_Code,Short_Code_2,Sap_Code,HSN_Code,Category_online_display,Attributes,Goods_Services,"GST% (Allowed only 5,12,18,28)",...,Addon_Group_Max,Addon_Group_Name,Addon_Group_Selection,Addon_Group_Min,Addon_Group_Max,Kcal,Carbs (g),Fat (g),Protien (g),Fiber
0,Curd Rice,,253,,,,Rice,veg,services,5,...,,,,,,207,37.6,3.2,6.1,1.8
1,Boiled Egg (1 Pcs),,204,,,,Egg Dry & Gravies,egg,services,5,...,,,,,,72,0.4,4.8,6.3,0


In [ ]:
cate.columns

Index(['Fixed Name ', 'Description', 'Short_Code', 'Short_Code_2', 'Sap_Code',
       'HSN_Code', 'Category_online_display', 'Attributes', 'Goods_Services',
       'GST% (Allowed only 5,12,18,28)',
       ...
       'Addon_Group_Max', 'Addon_Group_Name', 'Addon_Group_Selection',
       'Addon_Group_Min', 'Addon_Group_Max', 'Kcal', 'Carbs (g)', 'Fat (g)',
       'Protien (g)', 'Fiber'],
      dtype='object', length=233)

In [ ]:
db=db.reset_index()
#db.drop(columns=['Description'],inplace=True)
db.fillna(0,inplace=True)
db.columns=db.columns.droplevel

In [ ]:
table2.insert(0,'Description',value=''HSN_Code',',allow_duplicates=False)
table2=table2.reset_index() 
table2.fillna(0,inplace=True)

In [ ]:
# Create a sample DataFrame
data = {'name': ['John', 'Mary', 'Peter'], 'age': [25, 30, 35], 'city': ['New York', 'Paris', 'London']}
df = pd.DataFrame(data)
df

,name,age,city
0,John,25,New York
1,Mary,30,Paris
2,Peter,35,London


In [ ]:

# Iterate over each row of the DataFrame
for index, row in df.iterrows():
    print(f"Index: {index}, Name: {row['name']}, Age: {row['age']}, City: {row['city']}")


Index: 0, Name: John, Age: 25, City: New York
Index: 1, Name: Mary, Age: 30, City: Paris
Index: 2, Name: Peter, Age: 35, City: London


In [ ]:

dff = {'Customer':['A','B','C'],'Sale Amount':[123,545,744]}
df=pd.DataFrame(dff)

# create an empty dictionary to store the total revenue for each customer
customer_revenue = {}

# iterate over each row in the DataFrame
for index, row in df.iterrows():
    # get the customer name and the sale amount from the row
    customer_name = row['Customer']
    sale_amount = row['Sale Amount']
    # if the customer is not already in the dictionary, add them with the sale amount
    if customer_name not in customer_revenue:
        customer_revenue[customer_name] = sale_amount
    # if the customer is already in the dictionary, add the sale amount to their existing revenue
    else:
        customer_revenue[customer_name] += sale_amount

# print the total revenue for each customer
for customer, revenue in customer_revenue.items():
    print(f"{customer}: ${revenue}")


A: $123
B: $545
C: $744


In [ ]:
#@title original
worksheet = gc.open_by_key('1e3Orj6xVewBQiIMr0MAC9iJHy-UO-WxiGZDRaQcJyzE').sheet1
rows=worksheet.get_all_values()
menu=pd.DataFrame.from_records(rows[1:],columns=rows[0])

worksheet = gc.open_by_key('1NFCWTp1q9Q0LYhdBuIhyeYH1p0xcFrHBXjQZtEhME7M').worksheet('08_Menu_Item_Fixed_Name_V2')

rows=worksheet.get_all_values()
cate=pd.DataFrame.from_records(rows[1:],columns=rows[0])
# Load Table 1 data into a DataFrame
table1 = menu

# Load Table 2 data into a DataFrame
table2 = cate

# Loop over each row in Table 1
for index, row in table1.iterrows():
    # Check if the Online_Name from Table 1 is in the Fixed Name column of Table 2
    if row['Online_Name'] in table2['Fixed Name '].values:
        # If it is, get the corresponding row from Table 2
        matching_row = table2[table2['Fixed Name '] == row['Online_Name']]

        # Update the Description column of Table 1 with the matching Description value
        table1.at[index, 'Description'] = description

# Print the updated Table 1
#print(table1)
